<a href="https://colab.research.google.com/github/yan170506/Sistema-mercadinho../blob/main/mercadinho_corrigido0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import sqlite3
from datetime import datetime

# --- Configuração do Banco de Dados ---
conn = sqlite3.connect('mercadinho.db')
cursor = conn.cursor()

# Criar tabelas se não existirem
cursor.execute('''
CREATE TABLE IF NOT EXISTS produtos (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    categoria TEXT,
    preco REAL NOT NULL,
    quantidade INTEGER NOT NULL,
    data_cadastro TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS logs (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    acao TEXT NOT NULL,
    produto_id INTEGER,
    produto_nome TEXT,
    quantidade INTEGER,
    data_hora TEXT,
    FOREIGN KEY (produto_id) REFERENCES produtos (id)
)
''')

cursor.execute('''
CREATE TRIGGER IF NOT EXISTS log_insercao_produto
AFTER INSERT ON produtos
BEGIN
    INSERT INTO logs (acao, produto_id, produto_nome, quantidade, data_hora)
    VALUES ('INSERT', NEW.id, NEW.nome, NEW.quantidade, datetime('now'));
END;
''')

conn.commit()

# --- Funções Auxiliares ---
def input_com_cancelar(prompt, tipo=str, default=None):
    """
    Função para input com opção de cancelamento global.
    Retorna:
        - Valor digitado (convertido para o tipo especificado)
        - None se o usuário digitar '/cancelar'
    """
    while True:
        user_input = input(prompt)
        if user_input.strip().lower() == '/cancelar':
            return None
        try:
            if not user_input and default is not None:
                return default
            return tipo(user_input) if user_input else default
        except ValueError:
            print(f"Erro: Digite um valor válido de {tipo.__name__} ou '/cancelar' para cancelar.")

# --- Operações CRUD ---
def adicionar_produto():
    print("\n--- Adicionar Novo Produto (digite '/cancelar' a qualquer momento) ---")

    nome = input_com_cancelar("Nome do produto: ")
    if nome is None:
        print("❌ Operação cancelada.")
        return

    categoria = input_com_cancelar("Categoria: ")
    if categoria is None:
        print("❌ Operação cancelada.")
        return

    preco = input_com_cancelar("Preço: R$ ", float)
    if preco is None:
        print("❌ Operação cancelada.")
        return

    quantidade = input_com_cancelar("Quantidade em estoque: ", int)
    if quantidade is None:
        print("❌ Operação cancelada.")
        return

    try:
        cursor.execute('''
        INSERT INTO produtos (nome, categoria, preco, quantidade, data_cadastro)
        VALUES (?, ?, ?, ?, ?)
        ''', (nome, categoria, preco, quantidade, datetime.now().strftime("%Y-%m-%d %H:%M:%S")))

        conn.commit()
        print(f"✅ Produto '{nome}' adicionado com sucesso!")
    except Exception as e:
        print(f"❌ Erro ao adicionar produto: {e}")
        conn.rollback()

def listar_produtos():
    print("\n--- Lista de Produtos ---")
    cursor.execute('SELECT id, nome, categoria, preco, quantidade FROM produtos')
    produtos = cursor.fetchall()

    if not produtos:
        print("Nenhum produto cadastrado.")
    else:
        print("{:<5} {:<20} {:<15} {:<10} {:<10}".format(
            "ID", "Nome", "Categoria", "Preço", "Estoque"))
        print("-"*60)
        for prod in produtos:
            print("{:<5} {:<20} {:<15} R${:<8.2f} {:<10}".format(
                prod[0], prod[1], prod[2], prod[3], prod[4]))

def atualizar_produto():
    print("\n--- Atualizar Produto (digite '/cancelar' a qualquer momento) ---")
    listar_produtos()

    produto_id = input_com_cancelar("\nDigite o ID do produto a ser atualizado: ", int)
    if produto_id is None:
        print("❌ Operação cancelada.")
        return

    cursor.execute('SELECT * FROM produtos WHERE id = ?', (produto_id,))
    produto = cursor.fetchone()

    if not produto:
        print("❌ Produto não encontrado!")
        return

    print(f"\nEditando produto: {produto[1]}")

    nome = input_com_cancelar(f"Nome ({produto[1]}): ", default=produto[1])
    if nome is None:
        print("❌ Operação cancelada.")
        return

    categoria = input_com_cancelar(f"Categoria ({produto[2]}): ", default=produto[2])
    if categoria is None:
        print("❌ Operação cancelada.")
        return

    preco = input_com_cancelar(f"Preço (R$ {produto[3]}): ", float, default=produto[3])
    if preco is None:
        print("❌ Operação cancelada.")
        return

    quantidade = input_com_cancelar(f"Quantidade ({produto[4]}): ", int, default=produto[4])
    if quantidade is None:
        print("❌ Operação cancelada.")
        return

    try:
        cursor.execute('''
        UPDATE produtos
        SET nome=?, categoria=?, preco=?, quantidade=?
        WHERE id=?
        ''', (nome, categoria, preco, quantidade, produto_id))

        conn.commit()
        print("✅ Produto atualizado com sucesso!")
    except Exception as e:
        print(f"❌ Erro ao atualizar produto: {e}")
        conn.rollback()

def remover_produto():
    print("\n--- Remover Produto (digite '/cancelar' a qualquer momento) ---")
    listar_produtos()

    produto_id = input_com_cancelar("\nDigite o ID do produto a ser removido: ", int)
    if produto_id is None:
        print("❌ Operação cancelada.")
        return

    cursor.execute('SELECT nome FROM produtos WHERE id = ?', (produto_id,))
    produto = cursor.fetchone()

    if not produto:
        print("❌ Produto não encontrado!")
        return

    confirmacao = input(f"Tem certeza que deseja remover '{produto[0]}'? (s/n): ").lower()
    if confirmacao != 's':
        print("❌ Remoção cancelada.")
        return

    try:
        cursor.execute('DELETE FROM produtos WHERE id = ?', (produto_id,))
        conn.commit()
        print(f"✅ Produto '{produto[0]}' removido com sucesso!")
    except Exception as e:
        print(f"❌ Erro ao remover produto: {e}")
        conn.rollback()

def visualizar_logs():
    print("\n--- Logs do Sistema ---")
    cursor.execute('''
    SELECT l.id, l.acao, p.nome, l.quantidade, l.data_hora
    FROM logs l
    LEFT JOIN produtos p ON l.produto_id = p.id
    ORDER BY l.data_hora DESC
    ''')

    logs = cursor.fetchall()

    if not logs:
        print("Nenhum registro de log encontrado.")
    else:
        print("{:<5} {:<10} {:<20} {:<10} {:<20}".format(
            "ID", "Ação", "Produto", "Quantidade", "Data/Hora"))
        print("-"*70)
        for log in logs:
            print("{:<5} {:<10} {:<20} {:<10} {:<20}".format(
                log[0], log[1], log[2] or "N/A", log[3], log[4]))

# --- Menu Principal ---
def menu():
    print("\n=== SISTEMA DE ESTOQUE ===")
    print("1. Adicionar produto")
    print("2. Listar produtos")
    print("3. Atualizar produto")
    print("4. Remover produto")
    print("5. Visualizar logs")
    print("6. Sair")
    return input("Escolha uma opção: ")

def main():
    while True:
        opcao = menu()

        if opcao == '1':
            adicionar_produto()
        elif opcao == '2':
            listar_produtos()
        elif opcao == '3':
            atualizar_produto()
        elif opcao == '4':
            remover_produto()
        elif opcao == '5':
            visualizar_logs()
        elif opcao == '6':
            print("Saindo do sistema...")
            conn.close()
            break
        else:
            print("Opção inválida! Tente novamente.")

if __name__ == "__main__":
    main()


=== SISTEMA DE ESTOQUE ===
1. Adicionar produto
2. Listar produtos
3. Atualizar produto
4. Remover produto
5. Visualizar logs
6. Sair
Escolha uma opção: 1

--- Adicionar Novo Produto (digite '/cancelar' a qualquer momento) ---
Nome do produto: Beringela
Categoria: Horti
Preço: R$ 5
Quantidade em estoque: 55
✅ Produto 'Beringela' adicionado com sucesso!

=== SISTEMA DE ESTOQUE ===
1. Adicionar produto
2. Listar produtos
3. Atualizar produto
4. Remover produto
5. Visualizar logs
6. Sair
Escolha uma opção: 2

--- Lista de Produtos ---
ID    Nome                 Categoria       Preço      Estoque   
------------------------------------------------------------
1     Arroz                Grão            R$10.00    15        
3     Beringela            Horti           R$5.00     55        

=== SISTEMA DE ESTOQUE ===
1. Adicionar produto
2. Listar produtos
3. Atualizar produto
4. Remover produto
5. Visualizar logs
6. Sair
Escolha uma opção: 3

--- Atualizar Produto (digite '/cancelar' a qual